In [1]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=e2fe2c8a9c44b19156221861aec778677fb60bc909229c00aac54ca0d5a11572
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [4]:
csv_url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv'
df = pd.read_csv(csv_url)
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [3]:
import pandas as pd
import numpy as np

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkBy').getOrCreate()
apply_df = spark.createDataFrame(data=df)

In [7]:
apply_df.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|    

In [10]:



from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def capitalize_job(job):
  if job is not None:
    return job.capitalize()
  else:
    return job

capitalize_udf = udf(capitalize_job, StringType())

apply_df = apply_df.withColumn("Mjob_cap", capitalize_udf("Mjob")) \
                   .withColumn("Fjob_cap", capitalize_udf("Fjob"))

apply_df.show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------+--------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|Mjob_cap|Fjob_cap|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------+--------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|

In [14]:


# Create a function to print the last elements of the dataset
def print_last_elements(df, n=5):
  """
  Prints the last n elements of a Spark DataFrame.

  Args:
    df: The Spark DataFrame.
    n: The number of elements to print (default: 5).
  """
  last_elements = df.tail(n)
  for row in last_elements:
    print(row)

# Call the function to print the last 5 elements
print_last_elements(apply_df)



Row(school='MS', sex='M', age=20, address='U', famsize='LE3', Pstatus='A', Medu=2, Fedu=2, Mjob='services', Fjob='services', reason='course', guardian='other', traveltime=1, studytime=2, failures=2, schoolsup='no', famsup='yes', paid='yes', activities='no', nursery='yes', higher='yes', internet='no', romantic='no', famrel=5, freetime=5, goout=4, Dalc=4, Walc=5, health=4, absences=11, G1=9, G2=9, G3=9, Mjob_cap='Services', Fjob_cap='Services')
Row(school='MS', sex='M', age=17, address='U', famsize='LE3', Pstatus='T', Medu=3, Fedu=1, Mjob='services', Fjob='services', reason='course', guardian='mother', traveltime=2, studytime=1, failures=0, schoolsup='no', famsup='no', paid='no', activities='no', nursery='no', higher='yes', internet='yes', romantic='no', famrel=2, freetime=4, goout=5, Dalc=3, Walc=4, health=2, absences=3, G1=14, G2=16, G3=16, Mjob_cap='Services', Fjob_cap='Services')
Row(school='MS', sex='M', age=21, address='R', famsize='GT3', Pstatus='T', Medu=1, Fedu=1, Mjob='other', 

In [15]:


# Create a function to check if a person is of legal drinking age
def majority(age):
  if age > 17:
    return True
  else:
    return False

# Create a UDF for the majority function
majority_udf = udf(majority, StringType())

# Apply the UDF to create the new column
apply_df = apply_df.withColumn("legal_drinker", majority_udf("age"))

# Show the DataFrame with the new column
apply_df.show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------+--------+-------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|Mjob_cap|Fjob_cap|legal_drinker|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------+--------+-------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|   

In [17]:
# multiply by 10

from pyspark.sql.types import IntegerType, DoubleType, LongType, FloatType
from pyspark.sql.functions import col

def multiply_by_ten(df):
  """
  Multiplies every numeric column in a Spark DataFrame by 10.

  Args:
    df: The Spark DataFrame.
  """
  numeric_cols = [col_name for col_name in df.columns if df.schema[col_name].dataType in [
      IntegerType(), DoubleType(), LongType(), FloatType()
  ]]
  for col_name in numeric_cols:
    df = df.withColumn(col_name, col(col_name) * 10)
  return df

# Apply the function to the DataFrame
apply_df = multiply_by_ten(apply_df)

# Show the modified DataFrame
apply_df.show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------+--------+-------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|Mjob_cap|Fjob_cap|legal_drinker|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------+--------+-------------+
|    GP|  F|180|      U|    GT3|      A|  40|  40| at_home| teacher|    course|  mother|        20|       20|       0|      yes|    no|  no|        no|    yes|   yes|   